In [1]:
import os
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm

import copy
from statsmodels.formula.api import ols
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import LibrairiePerso_v4_8 as ownLibrary

import sys
sys.path.append("c:\python38\lib\site-packages")

#import importlib
#importlib.reload(ownLibrary)

import sklearn_pandas

pd.set_option('display.max_columns', 500)

In [2]:
#pip install statsmodels

In [3]:
import os
cwd = os.getcwd()


X_train = pd.read_csv(cwd + "\\data\\train_step_1.csv", sep=",")
y_train=X_train['SalePrice']

X_test = pd.read_csv(cwd + "\\data\\test_step_1.csv", sep=",")
y_test=X_test['SalePrice']

submission = pd.read_csv(cwd + "\\data\\submission_step_1.csv", sep=",")


for df in [X_train, X_test]:
    df.drop(['SalePrice'], axis=1, inplace=True)


# Quantitative features : Best transformation finder

In [4]:
target = 'SalePrice'
continuous_features = ['YrBltAndRemod','TotalSF','Total_sqr_footage','Total_Bathrooms','Total_porch_sf','LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch']

In [5]:
def transformationComparisonTable(X_train, X_test, y_train, y_test, scaleMethod) :

    scaled_train = ownLibrary.scale_features(X_train, [], scaleMethod)
    scaled_test = ownLibrary.scale_features(X_test, [], scaleMethod)
                
    
    scaled_dtr0 = ownLibrary.discretisationSupervise(scaled_train, scaled_test, y_train, y_test, False, False, 0)
    scaled_lr = ownLibrary.linearRegressionCorrelation(scaled_train, scaled_test, y_train, y_test)
    scaled_pr2 = ownLibrary.PolynomialFeaturesCorr(scaled_train, scaled_test, y_train, y_test, 2)
    scaled_pr3 = ownLibrary.PolynomialFeaturesCorr(scaled_train, scaled_test, y_train, y_test, 3)
    tableComparaison = pd.concat( [scaled_lr, scaled_dtr0, scaled_pr2, scaled_pr3], axis=1)
    
    return tableComparaison

In [6]:
result = pd.DataFrame()
for scaleMethod in [0,3,4]:
    df = transformationComparisonTable(X_train[continuous_features], X_test[continuous_features], y_train, y_test, scaleMethod)
    result = pd.concat( [result, df], axis=1)
    
result

,LR R²,DTR0_R²,PR2R²,PR3R²,LR R²,DTR0_R²,PR2R²,PR3R²,LR R²,DTR0_R²,PR2R²,PR3R²
YrBltAndRemod,31.26,35.47,35.59,35.69,31.26,34.79,35.60,35.70,31.26,34.79,35.60,35.70
TotalSF,66.63,63.34,63.66,68.81,66.63,61.72,63.27,68.84,66.63,61.72,63.27,68.84
Total_sqr_footage,56.96,50.58,54.60,57.97,56.96,47.82,54.28,57.90,56.96,47.82,54.28,57.90
Total_Bathrooms,41.64,39.74,41.83,41.38,41.64,39.74,41.82,41.39,41.64,39.74,41.82,41.39
Total_porch_sf,15.62,14.68,14.60,14.32,15.62,14.56,14.71,14.54,15.62,14.36,14.71,14.53
LotFrontage,12.21,14.72,12.72,12.95,12.21,14.68,12.68,12.83,12.21,14.68,12.68,12.83
LotArea,8.54,20.20,16.09,18.45,8.54,20.71,16.87,18.89,8.54,20.71,16.87,18.89
YearBuilt,24.97,36.31,34.58,34.39,24.97,35.79,34.64,34.44,24.97,35.79,34.64,34.44
YearRemodAdd,25.46,29.07,26.35,26.69,25.46,29.07,26.34,26.71,25.46,29.07,26.34,26.71
MasVnrArea,26.37,25.78,26.39,26.81,26.37,25.00,26.40,26.95,26.37,25.00,26.40,26.95


## Features/methods Best accuracy on target (scale method = 0)

In [7]:
dtr_best_performance = ['YrBltAndRemod', 'LotFrontage','LotArea','YearBuilt','YearRemodAdd','BsmtFinSF2','GarageYrBlt','WoodDeckSF','OpenPorchSF','EnclosedPorch']
pr2_best_performance = ['1stFlrSF','GarageArea']
pr3_best_performance = ['BsmtFinSF1','BsmtUnfSF','TotalBsmtSF','2ndFlrSF','GrLivArea']

#The accuracy gain is too small to perform a polynomial transformation
no_modification = ['MasVnrArea','ScreenPorch','TotalSF','Total_sqr_footage','Total_Bathrooms','Total_porch_sf']

## Bins list identified by Data Tree Regressor method

In [8]:
scaled_train0 = ownLibrary.scale_features(X_train, [], 0)
scaled_test0 = ownLibrary.scale_features(X_test, [], 0)
ownLibrary.discretisationSupervise(scaled_train0[dtr_best_performance], scaled_test0[dtr_best_performance], y_train, y_test, False, True, 0)

'YrBltAndRemod': {
	1: [0,3917],
	2: [3918,3966],
	3: [3967,4010],
	4: [4011,8040]
},
'LotFrontage': {
	1: [0,60.0],
	2: [61.0,74.0],
	3: [75.0,90.0],
	4: [91.0,220.0]
},
'LotArea': {
	1: [0,8635],
	2: [8640,10970],
	3: [10991,13680],
	4: [13682,430490]
},
'YearBuilt': {
	1: [0,1956],
	2: [1957,1984],
	3: [1985,2005],
	4: [2006,4020]
},
'YearRemodAdd': {
	1: [0,1956],
	2: [1957,1983],
	3: [1984,2007],
	4: [2008,4020]
},
'BsmtFinSF2': {
	1: [0,80],
	2: [81,1029],
	3: [1031,1057],
	4: [1063,2948]
},
'GarageYrBlt': {
	1: [0,1958.0],
	2: [1959.0,1987.0],
	3: [1988.0,2005.0],
	4: [2006.0,4020.0]
},
'WoodDeckSF': {
	1: [0,84],
	2: [85,96],
	3: [97,158],
	4: [159,1714]
},
'OpenPorchSF': {
	1: [0,8],
	2: [10,29],
	3: [30,43],
	4: [44,1046]
},
'EnclosedPorch': {
	1: [0,36],
	2: [37,42],
	3: [48,140],
	4: [143,1104]
},


,DTR0_R²
YrBltAndRemod,35.47
LotFrontage,14.72
LotArea,20.20
YearBuilt,36.31
YearRemodAdd,29.07
BsmtFinSF2,0.78
GarageYrBlt,34.53
WoodDeckSF,8.28
OpenPorchSF,21.29
EnclosedPorch,2.29


## Define features transformation based on Best transformation finder

## Quantitative features regroupement

If you set a new list of replacements you will probably have to fill manually the gaps between classes

In [9]:
#Left cap excluded, right cap included
ListeDesReglesQuantiKB = { 
    'YrBltAndRemod': {
        1: [0,3917],
        2: [3918,3966],
        3: [3967,4010],
        4: [4011,8040]
    },
    'LotFrontage': {
        1: [0,60.0],
        2: [61.0,74.0],
        3: [75.0,90.0],
        4: [91.0,220.0]
    },
    'LotArea': {
        1: [0,8639],
        2: [8640,10990],
        3: [10991,13680],
        4: [13682,430490]
    },
    'YearBuilt': {
        1: [0,1956],
        2: [1957,1984],
        3: [1985,2005],
        4: [2006,4020]
    },
    'YearRemodAdd': {
        1: [0,1956],
        2: [1957,1983],
        3: [1984,2007],
        4: [2008,4020]
    },
    'BsmtFinSF2': {
        1: [0,80],
        2: [81,1029],
        3: [1031,1061],
        4: [1063,2948]
    },
    'GarageYrBlt': {
        1: [0,1958.0],
        2: [1959.0,1987.0],
        3: [1988.0,2005.0],
        4: [2006.0,4020.0]
    },
    'WoodDeckSF': {
        1: [0,84],
        2: [85,96],
        3: [97,158],
        4: [159,1714]
    },
    'OpenPorchSF': {
        1: [0,8],
        2: [10,29],
        3: [30,43],
        4: [44,1046]
    },
    'EnclosedPorch': {
        1: [0,36],
        2: [37,47],
        3: [48,140],
        4: [143,1104]
    },
}



## Scale features

In [10]:
base_features = ['Id', 'MiscVal', 'SalePrice', 'SalePrice_log']
categorical_features_created = ['haspool','has2ndfloor','hasgarage','hasbsmt','hasfireplace']

scaleMethod0 = ['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','GarageYrBlt','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch']
scaleMethod1 = []
scaleMethod3 = []

ignore = base_features + pr2_best_performance + pr3_best_performance + no_modification + categorical_features_created

In [11]:
'''
if (scaleMethod1 !=[]): 
    X_train = ownLibrary.scale_features(X_train, scaleMethod1, 1)
    X_test = ownLibrary.scale_features(X_test, scaleMethod1, 1)
    submission = ownLibrary.scale_features(submission, scaleMethod1, 1)
'''

'''
if (scaleMethod3 !=[]): 
    X_train = ownLibrary.scale_features(X_train, scaleMethod3, 3)
    X_test = ownLibrary.scale_features(X_test, scaleMethod3, 3)
    submission = ownLibrary.scale_features(submission, scaleMethod3, 3)
'''

'\nif (scaleMethod3 !=[]): \n    X_train = ownLibrary.scale_features(X_train, scaleMethod3, 3)\n    X_test = ownLibrary.scale_features(X_test, scaleMethod3, 3)\n    submission = ownLibrary.scale_features(submission, scaleMethod3, 3)\n'

## Transformations

## Categorical replacements

In [12]:
ListeDesReglesQuali = { 

    'HeatingQC': {
        1: ["Ex"],
        2: ["Gd","TA"],
        3: ["Fa","Po"]
    },
    'CentralAir': {
        1: ["Y"],
        2: ["N"]
    },
    'Electrical': {
        1: ["SBrkr"],
        2: ["FuseA","FuseF","FuseP"],
        3: ["Mix","eNA"] 
    },
    'BsmtFullBath': {
        1: ["0"],
        2: ["1"],
        3: ["2","3"]
    },
    'BsmtHalfBath': {
        1: ["0"],
        2: ["1","2"]
    },
    'FullBath': {
        1: ["0","1"],
        2: ["2","3","4"]
    },
    'HalfBath': {
        1: ["0"],
        2: ["1"],
        3: ["2"]
    },
    'BedroomAbvGr': {
        1: ["0","1","2"],
        2: ["3"],
        3: ["4"],
        4: ["5","6","7","8"]
    },
    'KitchenAbvGr': {
        1: ["0","1"],
        2: ["2","3"]
    },
    'KitchenQual': {
        1: ["Ex"],
        2: ["Fa","Po","TA"],
        3: ["Gd"]
    },
    'TotRmsAbvGrd': {
        1: ["2","3"],
        2: ["4"],
        3: ["5","6","7","8","9"],
        4: ["10","11","12","13","14","15"]
    },
    'Functional': {
        1: ["Typ","Mod"],
        2: ["Min1","Min2"],
        3:["Maj1","Maj2","Sev"]
    },
    'Fireplaces': {
        1: ["0"],
        2: ["1","2","3","4"]
    },
    'FireplaceQu': {
        1: ["Ex"],
        2: ["Gd"],
        3: ["TA"],
        4: ["Fa","Po","eNA"]
    },
    'GarageType': {
        1: ["Attchd"],
        2: ["BuiltIn"],
        3: ["2Types","Detchd"],
        4: ["Basment"],
        5: ["CarPort", "eNA"]
    },
    'GarageFinish': {
        1: ["RFn","Fin"],
        2: ["Unf"],
        3: ["eNA"]
    },
    
    'GarageCars': {
        1: ["0"],
        2: ["1"],
        3: ["2"],
        4: ["3","4","5"]
    },
    
    'GarageQual': {
        1: ["TA","Gd"],
        2: ["Ex"],
        3: ["Fa","eNA","Po"]
    },
    'GarageCond': {
        1: ["TA"],
        2:  ["Gd"],
        3:  ["Fa","eNA"],
        4:  ["Po"],
        5:  ["Ex"]
    },
    'PavedDrive': {
        1: ["Y"],
        2: ["N","P"]
    },
    'Fence': {
        1: ["eNA"],
        2: ["MnPrv","GdWo","MnWw"],
        3: ["GdPrv"]
    },
    #'MiscFeature': {
    #    1: ["eNA"],
    #    2: ["Shed","Gar2","Othr","TenC"]
    #},
    'MoSold': {
        1: ["1","3","4","5","6","8","9","10","11","12"],
        2: ["7"],
        3: ["2"]
    },
    'YrSold': {
        1: ["2006"],
        2: ["2007"],
        3: ["2008"],
        4: ["2009"],
        5: ["2010"]
    },
    'SaleType': {
        1: ["WD","COD","ConLw","ConLI","ConLD","Oth","CWD","VWD","Con"],
        2: ["New"]
    },
    'SaleCondition': {
        1: ["Normal"],
        2: ["Abnorml","AdjLand"],
        3: ["Alloca","Family","Partial"]
    },
    'MSSubClass': {
        1: ["20","60","120"],
        2: ["30","45","160","190","40","180","90","150"],
        3: ["50","70"],
        4: ["75","80","85"]
    },
    'RoofStyle': {
        1: ["Hip"],
        2: ["Gable","Gambrel","Mansard","Flat","Shed"]
    },
    'Exterior1st': {
        1: ["CemntBd","VinylSd","BrkFace"],
        2: ["MetalSd","Wd Sdng","HdBoard","Plywood","Stucco","WdShing","AsbShng"],
        3: ["BrkComm","AsphShn","ImStucc","CBlock","Stone"]
    },
    'Exterior2nd': {
        1: ["CmentBd","VinylSd","BrkFace","ImStucc","Stucco","Stone"],
        2: ["MetalSd","Wd Sdng","HdBoard","Plywood","AsbShng","AsphShn","Wd Shng"],
        3: ["Brk Cmn","CBlock","Other"]
    },
    'MasVnrType': {
        1: ["None", "BrkCmn", "eNA"],
        2: ["Stone", "BrkFace"]
    },
    'ExterQual': {
        1: ["Ex","Gd"],
        2: ["TA","Fa"]
    },
    'ExterCond': {
        1: ["Ex","Gd","TA"],
        2: ["Fa","Po"]
    },
    'Foundation': {
        1: ["PConc","Wood","Stone"],
        2: ["CBlock","BrkTil","Slab"]
    },
    'BsmtQual': {
        1: ["Ex"],
        2: ["Gd","TA"],
        3: ["Fa","eNA"]
    },
    'BsmtCond': {
        1: ["TA"],
        2: ["Gd"],
        3: ["Fa","Po","eNA"]
    },
    'BsmtExposure': {
        1: ["No","Mn","Av"],
        2: ["Gd"],
        3: ["eNA"]
    },
    'BsmtFinType1': {
        1: ["GLQ"],
        2: ["ALQ","Rec","BLQ","LwQ","Unf","eNA"]
    },
    'BsmtFinType2': {
        1: ["Unf"],
        2: ["Rec","BLQ","LwQ","eNA"],
        3: ["ALQ","GLQ"]
    },

    'MSZoning': {
        1: ["A","I","RP"],
        2: ["FV","RL"],
        3: ["C (all)","RH","RM"]
    },
    'Alley': {
        1: ["Grvl","Pave"],
        2: ["eNA"]
    },
    'LotShape': {
        1: ["Reg"],
        2: ["IR1","IR2","IR3"]
    },
    'LandContour': {
        1: ["Lvl"],
        2: ["Bnk","HLS","Low"]
    },
    'LotConfig': {
        1: ["Inside"],
        2: ["Corner","FR2"],
        3: ["CulDSac","FR3"]
    },
    'LandSlope': {
        1: ["Gtl"],
        2: ["Mod","Sev"]
    },
    'Condition1': {
        1: ["Artery","Feedr","RRAe","RRAn","RRNe"],
        2: ["Norm"],
        3: ["PosA","PosN","RRNn"]
    },
    'BldgType': {
        1: ["1Fam"],
        2: ["2fmCon","Duplex","Twnhs"],
        3: ["TwnhsE"]
    },
    'HouseStyle': {
        1: ["1.5Fin","1.5Unf","2.5Unf","SFoyer","SLvl"],
        2: ["1Story","2.5Fin","2Story"]
    },
    'OverallQual': {
        1: ["1","2","3","4"],
        2: ["5"],
        3: ["6"],
        4: ["7","8","9","10"]
    },
    
    'Neighborhood' : {
        1: ["NoRidge","NridgHt","StoneBr"],
        2: ["CollgCr","Veenker","Crawfor","Somerst","NWAmes","SawyerW","Timber","Gilbert","ClearCr","Blmngtn"],
        3: ["Mitchel","OldTown","BrkSide","Sawyer","NAmes","IDOTRR","MeadowV","Edwards","NPkVill","BrDale","SWISU","Blueste"]
    },    
    'OverallCond': {
        1: ["1","2","3","4"],
        2: ["5"],
        3: ["6","7","8","9"]
    }
   
}

# Run regroupments

In [13]:
#X_train_rgrpd = copy.deepcopy(X_train)
#X_test_rgrpd = copy.deepcopy(X_test)
#submission_rgrpd = copy.deepcopy(submission)

In [14]:
#X_train_rgrpd

In [15]:
for dataset in [X_train, X_test, submission]: 
    # ----------- Pour chaque colonne du DataSet nous allons regrouper ------------
    for feature in dataset:      
        # ------------------------ If the feature is categorical ----------------------    
        if (feature in ListeDesReglesQuali):
            featureDiscretise = ownLibrary.discretise_1col_quali(dataset[feature], feature, ListeDesReglesQuali[feature])
            dataset.drop([feature], axis=1, inplace=True)
            dataset[feature] = featureDiscretise

        # ------------------------ If the feature is continuous ----------------------                            
        elif (feature in ListeDesReglesQuantiKB):
            featureDiscretise = ownLibrary.discretise_1col_quanti(dataset[feature], feature, ListeDesReglesQuantiKB[feature])
            if( len(dataset[feature]) != len(featureDiscretise)):
                print('len(dataset[feature]) != len(featureDiscretise) : ' + str(dataset[feature]) + ' != ' + str(featureDiscretise))
            else:    
                dataset.drop([feature], axis=1, inplace=True)
                dataset[feature] = featureDiscretise          
                    
     


In [16]:
X_train_rgrpd

NameError: name 'X_train_rgrpd' is not defined

# Features regroupment

In [ ]:

for df in [X_train, X_test, submission]:
    df["NewFirePlaces"] = df["Fireplaces"].astype(str) + df["FireplaceQu"].astype(str)
    df["NewExterQualCond"] = df["ExterQual"].astype(str) + df["ExterCond"].astype(str)
    df["NewCentrAirElec"] = df["CentralAir"].astype(str) + df["Electrical"].astype(str)
    df["NewKitchen"] = df["KitchenAbvGr"].astype(str) + df["KitchenQual"].astype(str)
    df["NewSale"] = df["SaleType"].astype(str) + df["SaleCondition"].astype(str)
    
    df.drop(['Fireplaces'], axis=1, inplace=True)
    df.drop(['FireplaceQu'], axis=1, inplace=True)
    df.drop(['ExterQual'], axis=1, inplace=True)
    df.drop(['ExterCond'], axis=1, inplace=True)
    df.drop(['CentralAir'], axis=1, inplace=True)
    df.drop(['Electrical'], axis=1, inplace=True)
    df.drop(['KitchenAbvGr'], axis=1, inplace=True)
    df.drop(['KitchenQual'], axis=1, inplace=True)
    df.drop(['SaleType'], axis=1, inplace=True)
    df.drop(['SaleCondition'], axis=1, inplace=True)
    

In [ ]:
X_train.head(5)

# Manage Spark

In [ ]:
#Export X_train, X_test, submission
X_train.loc[:, 'SalePrice']  = y_train
X_train.to_csv (cwd + "\\data\\train_spark_2.csv", index = False, header=True)
X_test.loc[:, 'SalePrice']  = y_test
X_test.to_csv (cwd + "\\data\\test_spark_2.csv", index = False, header=True)

submission['SalePrice'] = 0
submission.to_csv (cwd + "\\data\\submission_spark_2.csv", index = False, header=True)

In [ ]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
spark_train = spark.read.option("header", "true").option("delimiter", ",").csv(cwd + "\\data\\train_spark_2.csv")
spark_test = spark.read.option("header", "true").option("delimiter", ",").csv(cwd + "\\data\\test_spark_2.csv")

spark_submission = spark.read.option("header", "true").option("delimiter", ",").csv(cwd + "\\data\\submission_spark_2.csv")
#spark_submission.createOrReplaceTempView('spark_submission')

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer


from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
quantitative_features = pr2_best_performance + pr3_best_performance + no_modification
qualitative_features = [var for var in X_train.columns if var not in quantitative_features + base_features]

In [ ]:
from pyspark.sql.types import IntegerType

for feature in quantitative_features:
    spark_train = spark_train.withColumn(feature, spark_train[feature].cast(IntegerType()))
spark_train = spark_train.withColumn('SalePrice', spark_train['SalePrice'].cast(IntegerType()))

In [ ]:
indexed_features = [var+'_indexed' for var in qualitative_features ]
encoded_features = [var+'_ohe' for var in qualitative_features ]

indexer = StringIndexer()
indexer.setInputCols(qualitative_features)
indexer.setOutputCols(indexed_features)
indexed = indexer.fit(spark_train).transform(spark_train)

encoder = OneHotEncoder()
encoder.setInputCols(indexed_features)
encoder.setOutputCols(encoded_features)
encoded = encoder.fit(indexed).transform(indexed)

vectorAssembler = VectorAssembler(inputCols=quantitative_features+encoded_features, outputCol="quant_features")
features_vectorized = vectorAssembler.transform(encoded)

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [indexer, encoder, vectorAssembler])

transformed_df = pipeline.fit(spark_train).transform(spark_train)

transformed_df.createOrReplaceTempView('transformed_df')
#spark_train.show()
spark.sql("SELECT Id, quant_features, SalePrice FROM transformed_df limit 10").show()

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'quant_features', labelCol='SalePrice', maxIter=10, regParam=0.3, elasticNetParam=0.8)

lr_model = lr.fit(transformed_df)
#print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
#pipeline2 = Pipeline(stages=[indexer, encoder, vectorAssembler, lr])
#model = pipeline2.fit(spark_train)
#predictions = model.tranform(spark_train)

## Polynomial Transformation

In [ ]:
X_train = ownLibrary.PolynomialRegrTransformationReturnDF(X_train, pr2_best_performance, 2)
X_test = ownLibrary.PolynomialRegrTransformationReturnDF(X_test, pr2_best_performance, 2)
submission = ownLibrary.PolynomialRegrTransformationReturnDF(submission, pr2_best_performance, 2)


In [ ]:

X_train = ownLibrary.PolynomialRegrTransformationReturnDF(X_train, pr3_best_performance, 3)
X_test = ownLibrary.PolynomialRegrTransformationReturnDF(X_test, pr3_best_performance, 3)
submission = ownLibrary.PolynomialRegrTransformationReturnDF(submission, pr3_best_performance, 3)


In [ ]:
train_rwrk = copy.deepcopy(X_train)
test_rwrk = copy.deepcopy(X_test) 
submission_rwrk = copy.deepcopy(submission) 

## Export

In [28]:

X_train.loc[:, 'SalePrice']  = y_train
X_train.to_csv (cwd + "\\data\\train_step_2.csv", index = False, header=True)
X_test.loc[:, 'SalePrice']  = y_test
X_test.to_csv (cwd + "\\data\\test_step_2.csv", index = False, header=True)

submission['SalePrice'] = 0
submission.to_csv (cwd + "\\data\\submission_step_2.csv", index = False, header=True)